In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv('Dementia Prediction Dataset.csv')  # Or full file
print(df.shape)  # e.g., (num_rows, 570)
print(df['DEMENTED'].value_counts(normalize=True))  # Check imbalance (~20-30% dementia typical)

C:\Users\USER\AppData\Local\Temp\ipykernel_23064\4074265941.py:1: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Dementia Prediction Dataset.csv')  # Or full file


(195196, 1024)
DEMENTED
0    0.704963
1    0.295037
Name: proportion, dtype: float64


In [24]:
# --------------------------------------------------------------
# 1. REPLACE NACC SENTINEL CODES WITH np.nan
# --------------------------------------------------------------
special_codes = [-4, -4.4, 8, 9, 88, 888, 8888, 96, 996, -4.4, '']
df.replace(special_codes, np.nan, inplace=True)

In [25]:
# --------------------------------------------------------------
# 2. DERIVE AGE
# --------------------------------------------------------------
df['AGE'] = df['VISITYR'] - df['BIRTHYR']
assert (df['AGE'] >= 0).all(), "Negative AGE!"

In [28]:
# --------------------------------------------------------------
# 3. KEEP ONLY NON-MEDICAL FEATURES THAT ACTUALLY EXIST
# --------------------------------------------------------------

# Full list of non-medical columns we *want* (from Forms A1-A3, A5)
desired_features = [
    # A1: Subject
    'BIRTHMO','BIRTHYR','SEX','HISPANIC','HISPOR','HISPORX',
    'RACE','RACEX','RACESEC','RACESECX','RACETER','RACETERX',
    'PRIMLANG','PRIMLANX','EDUC','MARISTAT','NACCLIVS',
    'INDEPEND','RESIDENC','HANDED',
    # A2: Informant
    'INBIRMO','INBIRYR','INSEX','NEWINF','INHISP','INHISPOR','INHISPOX',
    'NACCNINR','INRACE','INRACEX','INRASEC','INRASECX','INRATER','INRATERX',
    'INEDUC','INRELTO','INRELTOX','INKNOWN','INLIVWTH','INVISITS','INCALLS','INRELY',
    # A3: Family History
    'NACCFAM','NACCMOM','NACCDAD','NACCAM','NACCAMX','NACCAMS','NACCAMSX',
    'NACCFM','NACCFMX','NACCFMS','NACCFMSX','NACCOM','NACCOMX','NACCOMS','NACCOMSX',
    'NACCFADM','NACCFFTD',
    # A5: Lifestyle
    'TOBAC30','TOBAC100','SMOKYRS','PACKSPER','QUITSMOK',
    'ALCOCCAS','ALCFREQ','ALCOHOL','ABUSOTHR','ABUSX',
    # Visit info
    'VISITMO','VISITDAY','VISITYR',
]

# Add target and ID
required_cols = ['DEMENTED', 'NACCID']

# Find which desired columns actually exist in the dataset
existing_features = [col for col in desired_features if col in df.columns]
missing_features = [col for col in desired_features if col not in df.columns]

print(f"Found {len(existing_features)} / {len(desired_features)} desired non-medical columns")
if missing_features:
    print(f"Missing columns (will be skipped): {missing_features}")

# Final column list: existing non-medical + target + ID
keep_cols = existing_features + required_cols

# Select only existing columns
df = df[keep_cols].copy()

print(f"Dataset reduced to {df.shape[1]} columns (non-medical + target + ID)")

Found 53 / 72 desired non-medical columns
Missing columns (will be skipped): ['SEX', 'HISPORX', 'RACEX', 'RACESECX', 'RACETERX', 'PRIMLANX', 'INSEX', 'INHISPOX', 'INRACEX', 'INRASECX', 'INRATERX', 'INRELTOX', 'NACCAMX', 'NACCAMSX', 'NACCFMX', 'NACCFMSX', 'NACCOMX', 'NACCOMSX', 'ABUSX']
Dataset reduced to 55 columns (non-medical + target + ID)


In [17]:
#DROP TEXT-EXTENSION COLUMNS (almost 100 % empty)
text_x_cols = [c for c in df.columns if c.endswith('X')]
df.drop(columns=text_x_cols, inplace=True)
print(f"Dropped {len(text_x_cols)} text-extension columns")

Dropped 19 text-extension columns


In [18]:
#MISSING-VALUE SUMMARY (helps decide imputation)
miss_perc = df.isnull().mean().round(4) * 100
miss_perc = miss_perc[miss_perc > 0].sort_values(ascending=False)
print("\n% missing per column (non-zero only):")
print(miss_perc)


% missing per column (non-zero only):
NACCOMS     99.82
NACCAMS     99.71
NACCFMS     99.50
INRATER     99.23
RACETER     98.73
INRASEC     98.50
INHISPOR    97.16
RACESEC     96.51
ALCFREQ     93.73
HISPOR      92.99
ALCOCCAS    90.10
INKNOWN     87.98
NACCAM      77.13
NACCOM      76.99
NACCFM      76.93
QUITSMOK    73.29
INHISP      69.13
INEDUC      68.99
INRACE      68.74
INVISITS    61.68
INCALLS     61.68
PACKSPER    38.66
SMOKYRS     37.50
TOBAC100    37.36
TOBAC30     36.97
ALCOHOL     36.92
ABUSOTHR    36.90
NEWINF      29.74
INBIRMO     20.80
BIRTHMO     17.56
VISITMO     16.58
NACCFAM      9.10
VISITDAY     7.09
INBIRYR      4.25
INRELTO      4.25
INRELY       4.25
INLIVWTH     4.25
NACCDAD      4.23
NACCNINR     3.28
NACCMOM      3.17
PRIMLANG     1.55
EDUC         1.48
RESIDENC     1.47
HANDED       0.53
MARISTAT     0.46
HISPANIC     0.35
INDEPEND     0.31
NACCLIVS     0.17
dtype: float64


In [31]:
# --------------------------------------------------------------
# 4. DERIVE AGE (safe)
# --------------------------------------------------------------
if 'VISITYR' in df.columns and 'BIRTHYR' in df.columns:
    df['AGE'] = df['VISITYR'] - df['BIRTHYR']
    print(f"AGE derived: min={df['AGE'].min()}, max={df['AGE'].max()}")
else:
    print("VISITYR or BIRTHYR missing → cannot derive AGE")

AGE derived: min=18, max=111


In [32]:
# --------------------------------------------------------------
# 5. DROP TEXT EXTENSION COLUMNS ('X')
# --------------------------------------------------------------
text_x_cols = [c for c in df.columns if c.endswith('X')]
df.drop(columns=text_x_cols, inplace=True)
print(f"Dropped {len(text_x_cols)} text-extension columns")

# Update allowed_features after dropping 'X'
allowed_features = [c for c in existing_features if c not in text_x_cols]
if 'AGE' in df.columns:
    allowed_features += ['AGE']

Dropped 0 text-extension columns


In [33]:
# --------------------------------------------------------------
# 6. FEATURE ENGINEERING
# --------------------------------------------------------------
if all(col in df.columns for col in ['TOBAC100', 'SMOKYRS', 'PACKSPER']):
    df['SMOKYRS'] = np.where(df['TOBAC100'] == 0, 0, df['SMOKYRS'])
    df['PACKSPER'] = np.where(df['TOBAC100'] == 0, 0, df['PACKSPER'])
    df['PACK_YEARS'] = df['SMOKYRS'] * df['PACKSPER']
    allowed_features.append('PACK_YEARS')

if 'AGE' in df.columns:
    df['AGE_GROUP'] = pd.cut(df['AGE'], bins=[0, 65, 80, 120],
                             labels=['<65', '65-80', '>80'], include_lowest=True)
    allowed_features.append('AGE_GROUP')

fam_cols = ['NACCMOM','NACCDAD','NACCAM','NACCFM','NACCOM']
fam_cols = [c for c in fam_cols if c in df.columns]
if fam_cols:
    df['FAM_DEM_COUNT'] = df[fam_cols].isin([1]).sum(axis=1)
    allowed_features.append('FAM_DEM_COUNT')

In [34]:
# --------------------------------------------------------------
# 7. IMPUTATION (safe)
# --------------------------------------------------------------
num_cols = df.select_dtypes(include=np.number).columns
num_cols = num_cols.drop([c for c in ['DEMENTED','NACCID'] if c in num_cols], errors='ignore')
for c in num_cols:
    df[c].fillna(df[c].median(), inplace=True)

cat_cols = df.select_dtypes(include='object').columns
cat_cols = cat_cols.drop([c for c in ['NACCID'] if c in cat_cols], errors='ignore')
for c in cat_cols:
    df[c].fillna(df[c].mode(dropna=True)[0], inplace=True)

print(f"After imputation: {df.isnull().sum().sum()} missing values")

C:\Users\USER\AppData\Local\Temp\ipykernel_23064\3003613901.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(df[c].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_23064\3003613901.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

After imputation: 0 missing values


C:\Users\USER\AppData\Local\Temp\ipykernel_23064\3003613901.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(df[c].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_23064\3003613901.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [35]:
# --------------------------------------------------------------
# 8. FINAL X, y
# --------------------------------------------------------------
df.dropna(subset=['DEMENTED'], inplace=True)
X = df[allowed_features].copy()
y = df['DEMENTED'].copy()

print(f"Final X: {X.shape}, y: {y.shape}")

Final X: (195196, 57), y: (195196,)


In [36]:
# save the full preprocessed dataframe (contains DEMENTED and NACCID)
out_path = 'Dementia_Prediction_preprocessed.csv'
df.to_csv(out_path, index=False)
print(f"Saved preprocessed dataframe to {out_path} with shape {df.shape}")

Saved preprocessed dataframe to Dementia_Prediction_preprocessed.csv with shape (195196, 59)
